In [5]:
import os
import re
import pandas as pd
from glob import glob

In [6]:
def process_bin_gene_counts(bin_dir, intersect_folder):
    gene_intersect_dir = os.path.join(bin_dir, "bedtools_out", intersect_folder)
    
    bed_files = glob(os.path.join(gene_intersect_dir, "*_genesIntersect.bed"))
    df_list = []
    
    for bedfile in bed_files:
        # Extract sample ID from filename (e.g., "A14891_125_155_genesIntersect.bed" -> "A14891_125_155")
        sample_id = os.path.basename(bedfile).replace("_genesIntersect.bed", "")
        
        print(f"Reading {sample_id}")
        df = pd.read_csv(
            bedfile,
            sep='\t',
            header=None,
            names=["chr", "source", "feature", "start", "end", "score", "strand", "phase", "attributes", "count"]
        )
        
        # Extract gene_id and gene_name from attributes
        df["gene_id"] = df["attributes"].apply(
            lambda x: re.search(r"gene:([^;]+)", x).group(1) if re.search(r"gene:([^;]+)", x) else None
        )
        df["gene_name"] = df["attributes"].apply(
            lambda x: re.search(r"Name=([^;]+)", x).group(1) if re.search(r"Name=([^;]+)", x) else None
        )
        
        # Retain only necessary columns
        df_final = df[["gene_id", "gene_name", "chr", "start", "end", "count"]].copy()
        df_final["sample"] = sample_id  # Attach sample info
        
        df_list.append(df_final)

    print("Finished reading files")
    print("Combining.....")

    # Combine all data into one DataFrame
    combined = pd.concat(df_list, ignore_index=True)
    
    print("Finished combining")

    # Pivot to wide format **keeping chr, start, end as gene locations**
    wide_counts = combined.pivot_table(
        index=["gene_id", "gene_name", "chr", "start", "end"], 
        columns="sample", 
        values="count",
        fill_value=0
    ).reset_index()

    return wide_counts

In [7]:
# Example usage
# bin_dir = "/Users/.../ctDNA_11042024/data/human_binned_sequences/125_155"
# counts_125_155 = process_bin_gene_counts(bin_dir)
# counts_125_155.to_csv("125_155_gene_counts.tsv", sep="\t", index=False)

# If you want to do the loop for each bin
bins = ["125_155","150_180","170_220","220_345","40_150"]
parent = "/Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences"
output_dir = "/Users/janzules/Roselab/ctDNA_11042024/data/human_binned_sequences/ctMatrices/"
#Which Counts?

In [8]:
intersect_type = 'gene'

if intersect_type == 'gene':
    intersect_folder = "geneIntersect"
    sub_folder = "geneCounts"
    out_name = "_gene_counts.tsv"
else:
    intersect_folder = "nuclIntersect"
    sub_folder = "nuclCounts"
    out_name = "_nucleosome_counts.tsv"

print(intersect_folder)

geneIntersect


In [9]:

for b in bins:
    bin_path = os.path.join(parent, b)
    mat = process_bin_gene_counts(bin_path, intersect_folder)
    outname = os.path.join(output_dir, sub_folder, f"{b}{out_name}")
    mat.to_csv(outname, sep="\t", index=False)

Reading A14902_125_155
Reading A14899_125_155
Reading A14895_125_155
Reading A14892_125_155
Reading A14905_125_155
Reading A14893_125_155
Reading A14904_125_155
Reading A14903_125_155
Reading A14898_125_155
Reading A14894_125_155
Reading A14906_125_155
Reading A14891_125_155
Reading A14896_125_155
Reading A14901_125_155
Reading A14897_125_155
Reading A14900_125_155
Finished reading files
Combining.....
Finished combining
Reading A14892_150_180
Reading A14905_150_180
Reading A14902_150_180
Reading A14899_150_180
Reading A14895_150_180
Reading A14903_150_180
Reading A14898_150_180
Reading A14894_150_180
Reading A14893_150_180
Reading A14904_150_180
Reading A14896_150_180
Reading A14901_150_180
Reading A14906_150_180
Reading A14891_150_180
Reading A14897_150_180
Reading A14900_150_180
Finished reading files
Combining.....
Finished combining
Reading A14904_170_220
Reading A14893_170_220
Reading A14898_170_220
Reading A14894_170_220
Reading A14903_170_220
Reading A14899_170_220
Reading A148

In [1]:
import os
import re
import pandas as pd
from glob import glob

def process_bin_gene_counts(bin_dir, intersect_folder):
    gene_intersect_dir = os.path.join(bin_dir, "bedtools_out", intersect_folder)
    
    bed_files = glob(os.path.join(gene_intersect_dir, "*_genesIntersect.bed"))
    df_list = []
    
    for bedfile in bed_files:
        # e.g. "A14891_125_155_genesIntersect.bed"
        sample_id = os.path.basename(bedfile).replace("_genesIntersect.bed","")  # "A14891_125_155"
        
        # Read the BED file
        # By default, pandas will treat # lines as comments if you have them. 
        # Adjust header=None if no header line in the file.
        print(f"Reading {sample_id}")
        df = pd.read_csv(
            bedfile,
            sep='\t',
            header=None,
            names=["chr","source","feature","start","end","score","strand","phase","attributes","count"]
        )
        
        # Parse gene_id and gene_name from the 'attributes' column
        # Look for strings like "ID=gene:ENSG00000186092;Name=OR4F5;..."
        # We'll capture "ENSG00000186092" and "OR4F5" with regex
        df["gene_id"] = df["attributes"].apply(
            lambda x: re.search(r"gene:([^;]+)", x).group(1) if re.search(r"gene:([^;]+)", x) else None
        )
        df["gene_name"] = df["attributes"].apply(
            lambda x: re.search(r"Name=([^;]+)", x).group(1) if re.search(r"Name=([^;]+)", x) else None
        )
        
        # Aggregate counts by gene
        df_sum = df.groupby(["gene_id","gene_name"], as_index=False)["count"].sum()
        
        # Attach sample info
        df_sum["sample"] = sample_id
        df_list.append(df_sum)

    print("Finished reading files")
    print("Combining.....")
    # Combine into one long df
    combined = pd.concat(df_list, ignore_index=True)
    print("Finished combining")
    # Pivot to wide format
    wide_counts = combined.pivot_table(
        index=["gene_id","gene_name"], 
        columns="sample", 
        values="count",
        fill_value=0
    ).reset_index()
    
    return wide_counts